# Run MTP on FABRIC Nodes

**A problem to solve**: The timing info is already done prior, so you would need to run this right away. Probably best to move that here instead once it's all ready.

## Input Required Information

In [ ]:
SLICE_NAME = "Clos"
NODE_TO_FAIL = "L1"
INTF_TO_FAIL = "eth1"

## Access the Slice

The orchestrator class is initalized, which also means the slice and its nodes are now accessable as well.

In [ ]:
from FabUtils import FabOrchestrator

try:
    manager = FabOrchestrator(SLICE_NAME)
    
except Exception as e:
    print(f"Exception: {e}")

## Delete the Log from a Prior Test if Necessary

In [ ]:
rmLogCmd = "rm MTP_*.log"
manager.executeCommandsParallel(rmLogCmd, prefixList="L,S")

## MTP **Initial Convergence**

Delay by a bit to get everything working first.

In [ ]:
import time

startHelloTime = round(time.time()*1000) + 3 * 60 * 1000

startCmd = f"tmux new-session -d -s mtp 'cd ~/SRC; sudo ./MTPstart -t {startHelloTime}'"

# First on Spines
manager.executeCommandsParallel(startCmd, prefixList="S")
print("Started MTP on spines")

# Wait a bit
time.sleep(5)

# Then on Leaves
manager.executeCommandsParallel(startCmd, prefixList="L")
print("Started MTP on Leaves")

# MTP **Reconvergence** Testing

## Take the Interface down

In [ ]:
# Take the specified interface down
intfName = f"{NODE_TO_FAIL}-{INTF_TO_FAIL}-p1"
intf = manager.slice.get_interface(intfName)

intf.ip_link_down()

## Stop MTP on Each Node

In [ ]:
stopCmd = "tmux kill-session -t mtp"
manager.executeCommandsParallel(stopCmd, prefixList="L,S")

## Collect Log Results

Now that the nodes have logged updates to their respective log files, they need to be downloaded to be analyzed.

In [ ]:
import os

LOG_DIR_PATH = "../logs"
LOG_NAME = "MTP_{name}.log"
logPath = os.path.join(LOG_DIR_PATH, LOG_NAME)

# If the logs directory does not already exist, create it
if not os.path.exists(LOG_DIR_PATH):
    os.makedirs(LOG_DIR_PATH)
    
manager.downloadFilesParallel(logPath, LOG_NAME, prefixList="L,S", addNodeName=True)

## Bring the Interface Back Up

In [ ]:
intf.ip_link_up()